In [ ]:
import numpy as np
import torch
from kuramoto import Kuramoto
from matplotlib import pyplot as plt

# Kuramoto Dataset

In [ ]:
cluster_size = 4
n_clusters = 2
n_timesteps = 300

In [ ]:
def create_adjacency_matrix():
    cluster_blocks = []
    class_blocks = []
    for n in range(0, n_clusters):
        cluster_blocks.append(torch.ones([cluster_size, cluster_size]))
        class_blocks.append(torch.ones(cluster_size, dtype=torch.int64) * n)
    single_sample_adj = torch.block_diag(*cluster_blocks)
    return single_sample_adj

In [ ]:
def create_samples(adj=None):
    if adj is None:
        single_sample_adj = create_adjacency_matrix()
    else:
        single_sample_adj = adj

    nat_freqs = np.random.normal(loc=10, scale=1.0, size=n_clusters * cluster_size)
    model = Kuramoto(
        coupling=10,
        dt=0.001,
        T=n_timesteps * 2 * 0.001,  # double timesteos to cover train and val/test ranges
        n_nodes=len(single_sample_adj.numpy()),
        natfreqs=nat_freqs,
    )

    act_mat = model.run(
        adj_mat=single_sample_adj.numpy(),
        angles_vec=np.random.uniform(0, 2 * np.pi, size=n_clusters * cluster_size),
    )

    return single_sample_adj, act_mat

In [ ]:
adj, act_mat = create_samples()

In [ ]:
def plot_activity_clusters_unique(activities):
    """
    Plot sin(angle) vs time for each oscillator time series.
    activity: 2D-np.ndarray
        Activity time series, node vs. time; ie output of Kuramoto.run()
    return:
        matplotlib axis for further customization
    """
    # colors = ['tab_blue', 'tab_orange']
    colors = plt.cm.rainbow(np.linspace(0, 1, 8))
    linestyles = [
        "dashed",
        "dashed",
        "dashed",
        "dashed",
        "dashdot",
        "dashdot",
        "dashdot",
        "dashdot",
    ]
    print(colors)
    plt.clf()

    fig, ax0 = plt.subplots(1, 1, figsize=(10, 5))
    for activity, color, linestyle in zip(activities, colors, linestyles):
        ax0.plot(np.sin(activity.T), c=color, linestyle=linestyle, linewidth=3)
        ax0.set_xlabel("Step")
        ax0.set_ylabel(r"$\sin(\theta)$")

    ax0.legend(["0", "1", "2", "3", "4", "5", "6", "7"], bbox_to_anchor=(1.15, 1.0))

    return fig

In [ ]:
plt.rcParams.update({"font.size": 15})

In [ ]:
plot_activity_clusters_unique(act_mat)

Now we shuffle the adjacency to create test data

In [ ]:
# Shuffling rows of the adjacency matrix
r = np.array([7, 1, 2, 3, 4, 5, 6, 0])
c = np.array([7, 1, 2, 3, 4, 5, 6, 0])
shuffled_adj = adj[r[:, None], c]  # shuffles rows
shuffled_adj = adj[r][:, c]  # shuffles columns

In [ ]:
shuffled_adj, act_mat_shuffled = create_samples(adj=shuffled_adj)

In [ ]:
plot_activity_clusters_unique(act_mat_shuffled)

Let's visualize the adjacency matrices

In [ ]:
plt.imshow(adj)
plt.show()

In [ ]:
def show_adjacency_matrix(adj):
    fig, ax = plt.subplots()
    ax.imshow(adj, cmap="Greys")

    # draw gridlines
    ax.grid(which="major", axis="both", linestyle="-", color="grey", linewidth=2)
    ax.set_xticks(np.arange(-0.5, 8, 1))
    ax.set_yticks(np.arange(-0.5, 8, 1))

    xticklabelslabels = [
        "           0",
        "           1",
        "           2",
        "           3",
        "           4",
        "           5",
        "           6",
        "           7",
        "",
    ]

    yticklabelslabels = [
        "\n\n\n0",
        "\n\n\n1",
        "\n\n\n2",
        "\n\n\n3",
        "\n\n\n4",
        "\n\n\n5",
        "\n\n\n6",
        "\n\n\n7",
        "",
    ]

    ax.set_xticklabels(xticklabelslabels)
    ax.set_yticklabels(yticklabelslabels)

    ax.xaxis.tick_top()

    plt.show()

In [ ]:
show_adjacency_matrix(adj)

In [ ]:
show_adjacency_matrix(shuffled_adj)

# Plot in same style as SWaT analyses

In [ ]:
from string import ascii_letters

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

sns.set_theme(style="white")

In [ ]:
# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 11))

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(
    adj,
    mask=np.eye(8),
    cmap="viridis",
    square=True,
    linewidths=0.5,
    cbar_kws={"shrink": 0.5},
    linecolor="grey",
    clip_on=False,
    cbar=True,
    vmin=0,
    vmax=1,
)

In [ ]:
# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 11))

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(
    shuffled_adj,
    mask=np.eye(8),
    cmap="viridis",
    square=True,
    linewidths=0.5,
    cbar_kws={"shrink": 0.5},
    linecolor="grey",
    clip_on=False,
    cbar=True,
    vmin=0,
    vmax=1,
)

# Kuramoto correlation matrix

In [ ]:
seed = 3
kuramoto_corr = torch.load(f"../data/kuramoto/{seed}_2_4/correlation_matrix.pt").numpy()

# Min max scaling as done for calculating the adjacency error
kuramoto_corr = (kuramoto_corr - kuramoto_corr.min()) / (kuramoto_corr.max() - kuramoto_corr.min())

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 11))

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(
    kuramoto_corr,
    mask=np.eye(8),
    cmap="viridis",
    square=True,
    linewidths=0.5,
    cbar_kws={"shrink": 0.5},
    linecolor="grey",
    clip_on=False,
    cbar=True,
)

# Kuramoto training results

In [ ]:
kuramoto_static_ep50_test = pd.read_csv(
    "vis_samples/kuramoto/kuramoto_static_ep50_test.csv", index_col=0
)

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 11))

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(
    kuramoto_static_ep50_test,
    mask=np.eye(8),
    cmap="viridis",
    square=True,
    linewidths=0.5,
    cbar_kws={"shrink": 0.5},
    linecolor="grey",
    clip_on=False,
    cbar=True,
)

In [ ]:
kuramoto_dynamic_ep50_val_sample_1 = pd.read_csv(
    "vis_samples/kuramoto/kuramoto_dynamic_ep50_val_sample_1.csv", index_col=0
)

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 11))

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(
    kuramoto_dynamic_ep50_val_sample_1,
    mask=np.eye(8),
    cmap="viridis",
    square=True,
    linewidths=0.5,
    cbar_kws={"shrink": 0.5},
    linecolor="grey",
    clip_on=False,
    cbar=True,
)

In [ ]:
kuramoto_dynamic_ep50_val_sample_2 = pd.read_csv(
    "vis_samples/kuramoto/kuramoto_dynamic_ep50_val_sample_2.csv", index_col=0
)

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 11))

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(
    kuramoto_dynamic_ep50_val_sample_2,
    mask=np.eye(8),
    cmap="viridis",
    square=True,
    linewidths=0.5,
    cbar_kws={"shrink": 0.5},
    linecolor="grey",
    clip_on=False,
    cbar=True,
)

In [ ]:
kuramoto_dynamic_ep50_test_sample_1 = pd.read_csv(
    "vis_samples/kuramoto/kuramoto_dynamic_ep50_test_sample_1.csv", index_col=0
)

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 11))

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(
    kuramoto_dynamic_ep50_test_sample_1,
    mask=np.eye(8),
    cmap="viridis",
    square=True,
    linewidths=0.5,
    cbar_kws={"shrink": 0.5},
    linecolor="grey",
    clip_on=False,
    cbar=True,
)